In [1]:
import numpy as np
import pandas as pd

In [9]:
eps = 0.001
alpha, beta = 0.1*13, 0.1*13
N = 4
A = np.array([
    [10 + alpha, -1, 0.2, 2],
    [1, 12-alpha, -2, 0.1],
    [0.3, -4, 12-alpha, 1],
    [0.2, -0.3, -0.5, 8-alpha]
])
Adiag = np.diagonal(A)
B = np.array([1+beta, 2-beta, 3, 1])

F = np.copy(A)
for i in range(N):
    F[i,i] = 0
for i in range(N):
    for j in range(N):
        F[i,j] = F[i,j]/Adiag[i]
Fup = np.copy(F)
Fdown = np.copy(F)
for i in range(N):
    for j in range(N):
        if j > i:
            Fdown[i,j] = 0
        else:
            Fup[i,j] = 0
        
C = np.copy(B)
for i in range(N):
    C[i] = C[i]/Adiag[i]

[[ 0.         -0.08849558  0.01769912  0.17699115]
 [ 0.09345794  0.         -0.18691589  0.00934579]
 [ 0.02803738 -0.37383178  0.          0.09345794]
 [ 0.02985075 -0.04477612 -0.07462687  0.        ]]
[[ 0.         -0.08849558  0.01769912  0.17699115]
 [ 0.          0.         -0.18691589  0.00934579]
 [ 0.          0.          0.          0.09345794]
 [ 0.          0.          0.          0.        ]]
[[ 0.          0.          0.          0.        ]
 [ 0.09345794  0.          0.          0.        ]
 [ 0.02803738 -0.37383178  0.          0.        ]
 [ 0.02985075 -0.04477612 -0.07462687  0.        ]]


In [10]:
def matrixMultByVector(A, x):
    return np.array([ 
        sum([ A[i,j]*x[j] for j in range(N) ]) for i in range(N)
    ])

In [11]:
def simpleNextIter(x):
    res = np.zeros(N)
    for i in range(N):
        res[i] = C[i] - matrixMultByVector(Fdown, res)[i] - matrixMultByVector(Fup, x)[i] 
    return res

def norm(x):
    return np.max(np.abs(x))

def countDelta(xOld, xNew):
    deltaBig = findFNorm()/(1-findFNorm())*norm(xOld-xNew)
    deltaSmall = deltaBig/norm(xNew)
    return deltaBig, deltaSmall
    return any(map(lambda x: x < eps, delta))

def checkStop(xOld, xNew):
    return norm(xOld-xNew) < eps
        
def findFNorm():
    return max( np.sum(np.abs(F[i])) for i in range(N) )

In [12]:
def sipleIterations():
    df = pd.DataFrame({
        'DELTA': [],
        'delta': [],
    })
    xOld = C
    xNew = simpleNextIter(xOld)
    i = 0
    DELTA, delta = simpleCountDelta(xOld = xOld, xNew = xNew)
    app = { 
        'delta': delta,
        'DELTA': DELTA
    }
    df = df.append(app, ignore_index=True)
    
    while not simpleCheckStop(xOld,xNew):
        i += 1
        xNew, xOld = simpleNextIter(xNew), xNew
        DELTA, delta = simpleCountDelta(xOld = xOld, xNew = xNew)
        app = { 
            'delta': delta,
            'DELTA': DELTA
        }
        df = df.append(app, ignore_index=True)
    print(df)
    return xNew, delta
    
def showSimpleIterations():
    xNew, delta = sipleIterations()
    check = matrixMultByVector(A, xNew) 
    print("CHECK:", check)
    print("WANT TO GET", B)
    print("DIFFERENCE", check - B)
    print("NORM F", findFNorm())

In [13]:
showSimpleIterations()

      DELTA     delta
0  0.033744  0.112952
1  0.003272  0.010981
2  0.000280  0.000940
CHECK: [2.30010388 0.70016017 2.99997808 1.        ]
WANT TO GET [2.3 0.7 3.  1. ]
DIFFERENCE [ 1.03882649e-04  1.60170537e-04 -2.19202548e-05  0.00000000e+00]
NORM F 0.4953271028037384


/tmp/ipykernel_3430561/3951325670.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_3430561/3951325670.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_3430561/3951325670.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
